In [2]:
import os

In [1]:
pwd

'f:\\projects\\project-4 chicken disease predictor\\cd predict\\chicken-project\\reasearch'

In [3]:
os.chdir('../')

In [4]:
pwd

'f:\\projects\\project-4 chicken disease predictor\\cd predict\\chicken-project'

In [23]:
import tensorflow as tf
model = tf.keras.models.load_model('artifacts/training/model.h5')

[2025-12-03 05:28:42,154: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]


In [6]:
from dataclasses import dataclass
from  pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: int
    params_batch_size: list

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfiguraionManager:
    def __init__(self,
                 config_filepath =CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE,
        )
        return eval_config
     

In [11]:
from urllib.parse import urlparse

In [26]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        
    def _valid_generator(self):
        
        datagenrator_kwargs = dict(
            rescale=1./255,
            validation_split=0.30
        )
        dataflow_kwargs = dict(
            target_size=(self.config.params_image_size[:-1]),
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenrator_kwargs)
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
        
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        
    def save_score(self):
        scores = {
            "loss": self.score[0],
            "accuracy": self.score[1]
        }
        save_json(
            path=Path("scores.json"),
            data=scores
        )

In [27]:
try:
    config = ConfiguraionManager()
    eval_config = config.get_validation_config()
    evaluation = Evaluation(config=eval_config)
    evaluation.evaluation()
    evaluation.save_score()
except Exception as e:
    raise e

[2025-12-03 05:29:38,328: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-12-03 05:29:38,341: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-03 05:29:38,345: INFO: common: created directory at : artifacts]


[2025-12-03 05:29:39,029: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 15 images belonging to 2 classes.
1/1 ━━━━━━━━━━━━━━━━━━━━ 19s 19s/step - accuracy: 0.6000 - loss: 22.6025
[2025-12-03 05:30:03,312: INFO: common: json file saved at:scores.json]
